# Streaming

In [16]:
import asyncio
from openai.types.responses import ResponseTextDeltaEvent
from agents import Agent, Runner
from dotenv import load_dotenv
import nest_asyncio

load_dotenv()
nest_asyncio.apply()

async def main():
    agent = Agent(
        name="Joker",
        instructions="You are a helpful assistant.",
    )

    result = Runner.run_streamed(agent, input="Please tell me 5 jokes.")
    async for event in result.stream_events():
        if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
            print(event.data.delta, end="", flush=True)

# Jupyter Notebook veya async ortam için:
await main()


Sure, here are five jokes for you:

1. **Why don't scientists trust atoms?**
   - Because they make up everything!

2. **Why did the scarecrow win an award?**
   - Because he was outstanding in his field!

3. **What do you call fake spaghetti?**
   - An impasta!

4. **What did one wall say to the other wall?**
   - I'll meet you at the corner!

5. **Why did the bicycle fall over?**
   - Because it was two-tired!

In [17]:
import asyncio
import random
from agents import Agent, ItemHelpers, Runner, function_tool

@function_tool
def how_many_jokes() -> int:
    return random.randint(1, 10)


async def main():
    agent = Agent(
        name="Joker",
        instructions="First call the `how_many_jokes` tool, then tell that many jokes.",
        tools=[how_many_jokes],
    )

    result = Runner.run_streamed(
        agent,
        input="Hello",
    )
    print("=== Run starting ===")

    async for event in result.stream_events():
        # We'll ignore the raw responses event deltas
        if event.type == "raw_response_event":
            continue
        # When the agent updates, print that
        elif event.type == "agent_updated_stream_event":
            print(f"Agent updated: {event.new_agent.name}")
            continue
        # When items are generated, print them
        elif event.type == "run_item_stream_event":
            if event.item.type == "tool_call_item":
                print("-- Tool was called")
            elif event.item.type == "tool_call_output_item":
                print(f"-- Tool output: {event.item.output}")
            elif event.item.type == "message_output_item":
                print(f"-- Message output:\n {ItemHelpers.text_message_output(event.item)}")
            else:
                pass  # Ignore other event types

    print("=== Run complete ===")


await main()

=== Run starting ===
Agent updated: Joker
-- Tool was called
-- Tool output: 2
-- Message output:
 Here are two jokes for you:

1. Why don't scientists trust atoms?  
   Because they make up everything!

2. What do you call fake spaghetti?  
   An impasta!
=== Run complete ===
